# Proyecto ML Taxi. NB1: Limpieza de datos y creación de dataframe

### En este notebook se describe el proceso de limpieza y generación del dataframe final con el que se trabajarán los modelos de ML.

### Obtención del dataset
Se carga dataframe con los datos recuperados de (https://huggingface.co/datasets/Chendi/NYC_TAXI_FARE_CLEANED)

In [3]:
# Importa la función 'load_dataset' de la librería 'datasets', que permite cargar datasets de Hugging Face.
from datasets import load_dataset  
# Carga el dataset "Chendi/NYC_TAXI_FARE_CLEANED" desde Hugging Face en su formato predeterminado (diccionario de datasets)
df = load_dataset("Chendi/NYC_TAXI_FARE_CLEANED")  
# Convierte el dataset de Hugging Face (que está en formato 'DatasetDict') a un DataFrame de pandas.
df = df["train"].to_pandas()  

### Limpieza de datos 

In [4]:
#Se eliminan valores nulos
from functions_limpieza import limpieza1
df=limpieza1(df)

Se eliminaron 335 por valores nulos. Restan 54315620 datos.


### Reducción del dataframe 
Dada la cantidad de datos original, se decide trabajar con una muestra menor.

In [5]:
df_sample=df.sample(100000) #Se toma una muestra de 100k de datos del data originial.

### Feature engineering
A continuación se crean algunas varibles nuevas, provenientes de los datos base. Importante tener en cuenta que no se generan variables relacionadas con la tarifa del viaje, dado que esa es la característica a predecir.

In [6]:
from functions_limpieza import ft_eng1
df = ft_eng1(df_sample) #se aplica función para añadir columnas con variables calculadas a partir de las variables iniciales


 Se eliminó la columna 'pickup_datetime'. Las columnas en el dataframe son: Index(['fare_amount', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
       'pickup_hour', 'pickup_minutes', 'peak_hour', 'pickup_year',
       'pickup_month', 'pickup_day', 'day_of_week', 'distance'],
      dtype='object').


In [7]:
#Visualización del dataframe generado
df.sample(5)

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour,pickup_minutes,peak_hour,pickup_year,pickup_month,pickup_day,day_of_week,distance
5400865,22.50,-73.983251,40.721217,-73.858787,40.737257,1,22,27,False,2012,2,5,6,10.638371
36003405,11.70,-73.994738,40.760397,-74.008712,40.724212,1,12,44,False,2011,2,19,5,4.192282
47883915,57.33,-73.991203,40.747879,-73.990829,40.747971,1,14,11,False,2015,1,5,0,0.033097
51362474,5.70,-73.942700,40.790260,-73.956083,40.769568,5,23,35,False,2010,7,22,3,2.561964
21132601,5.70,-74.005248,40.740968,-73.991990,40.735388,1,12,35,False,2011,4,15,4,1.277775


Las variables creadas involucran datos cícilicos temporales como horas, minutos, meses y días. Por esta razón y con mira en el análisis de machine learning, se aplicará una transformación trigonométrica. Como resultado, cada variable será representada en un espacio bidimensional por medio de valores de senos y cosenos.

In [8]:
#se define lista con las columnas que representan variables cíclicas
variables_ciclicas=['pickup_hour', 'pickup_minutes','pickup_month', 'pickup_day', 'day_of_week']

from functions_limpieza import ft_eng2
df_final=ft_eng2(df,variables_ciclicas) # Se aplica función para transformación de variables cíclicas

In [9]:
#Visualización del dataframe generado
df_final.sample(5)

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,peak_hour,pickup_year,distance,pickup_hour_sin,pickup_hour_cos,pickup_minutes_sin,pickup_minutes_cos,pickup_month_sin,pickup_month_cos,pickup_day_sin,pickup_day_cos,day_of_week_sin,day_of_week_cos
53475545,6.9,-74.001419,40.751460,-74.001444,40.751462,1,False,2010,0.002118,5.195840e-01,0.854419,0.638244,-0.769834,5.000000e-01,-0.866025,0.201299,0.979530,-8.660254e-01,0.5
40783887,23.3,-73.987240,40.733139,-73.868078,40.722865,1,False,2010,10.105993,0.000000e+00,1.000000,-0.364161,-0.931336,8.660254e-01,-0.500000,0.897805,-0.440394,-8.660254e-01,0.5
53933794,24.9,-73.869077,40.771335,-73.950917,40.777597,1,True,2011,6.926537,-9.976688e-01,-0.068242,0.638244,-0.769834,-5.000000e-01,-0.866025,-0.299363,-0.954139,-2.449294e-16,1.0
30586896,18.5,-73.957545,40.765718,-74.001097,40.746572,1,False,2009,4.241379,-2.449294e-16,1.000000,0.053222,-0.998583,-2.449294e-16,1.000000,0.651372,-0.758758,-8.660254e-01,0.5
7456352,2.5,-73.982010,40.758026,-73.982018,40.758034,1,False,2015,0.001064,-7.308360e-01,0.682553,-0.314077,0.949398,5.000000e-01,0.866025,0.651372,-0.758758,0.000000e+00,1.0


### Creación de datafreme final en archivo csv

In [10]:
#Conversión del dataframe final a archivo csv
df_final.to_csv('data/df_final.csv', index=False, encoding='utf-8')